In [8]:
import json
import pandas as pd
import os

schedulers = ['CFS', 'FIFO', 'RR', 'RM']
column_names = []
exp_result_dir = os.path.join(os.getcwd(), '../exp_results')

data = {}

# Load data from JSON files
for scheduler in schedulers:
    file_path = os.path.join(exp_result_dir, f'{scheduler}-phased_result.json')
    with open(file_path, 'r') as file:
        results = json.load(file)
        data[scheduler] = results

# Extract column names
column_names.extend(["Task", "Core_index", "DeadLine(us)", "WCET(us)"])
column_names.extend([f"{scheduler}_Sched" for scheduler in schedulers])
column_names.extend([f"{scheduler}_WCRT(us)" for scheduler in schedulers])
column_names.extend([f"{scheduler}_AVG(us)" for scheduler in schedulers])

In [9]:
# Initialize an empty list to store data
data_rows = []

def find_task_info_by_name(task_name, scheduler):
    for info in data[scheduler]:
        if info['task_name'] == task_name:
            return info

# Fill in the list with task information
for task in data[schedulers[0]]:
    row_data = {}
    task_name = task['task_name']
    row_data['Task'] = task_name
    row_data['Core_index'] = task['core_index']
    row_data['DeadLine(us)'] = int(task['deadline_ns'] / 1000)
    row_data['WCET(us)'] = int(task['wcet_ns'] / 1000)
    for scheduler in schedulers:
        task_info = find_task_info_by_name(task_name, scheduler)
        row_data[f"{scheduler}_WCRT(us)"] = int(task_info['wcrt_ns'] / 1000)
        row_data[f"{scheduler}_AVG(us)"] = int(task_info['avg_response_time_ns'] / 1000)
        if task_info['wcrt_ns'] < task_info['deadline_ns']:
            row_data[f"{scheduler}_Sched"] = "Yes"
        else:
            row_data[f"{scheduler}_Sched"] = "No"
    data_rows.append(row_data)

# Create DataFrame from the list of data
analysis_result = pd.DataFrame(data_rows, columns=column_names)

In [10]:
analysis_result

,Task,Core_index,DeadLine(us),WCET(us),CFS_Sched,FIFO_Sched,RR_Sched,RM_Sched,CFS_WCRT(us),FIFO_WCRT(us),RR_WCRT(us),RM_WCRT(us),CFS_AVG(us),FIFO_AVG(us),RR_AVG(us),RM_AVG(us)
0,Task_4000ms,2,4000000,2000000,Yes,Yes,No,No,3980166,3270146,4292521,5400311,3980166,3270146,4292521,5400311
1,Task_2000ms,2,2000000,990000,Yes,No,No,Yes,1970105,2180073,2274138,1391609,1954086,1597552,2150051,1127976


In [11]:
# cwd = os.getcwd()
# print(cwd)
# assert False
import numpy as np

tasks_info_file = "../../../dataset/FMTV_application/generated_FMTV_json/tasks_info.json"
tasks_info = json.load(open(tasks_info_file, 'r'))

column_names = ["name", "core", "period", "wcet", "read", "execution", "write"]

data_rows = []
for task in tasks_info:
        row_data = {}
        row_data["name"] = task["task_name"]
        if task["isPeriodic"]:
                row_data["period"] = int(task["period_ns"] / 1000)
        else:
                row_data["period"] = int(task["lower_bound_ns"] / 1000)
        row_data["core"] = task["core_index"]
        row_data["read"] = int(task["phased_read"] / 1000)
        row_data["write"] = int(task["phased_write"] / 1000)
        phased_execution = task["phased_execution"]
        phased_execution = np.array(phased_execution) / 1000
        row_data["execution"] = np.max(phased_execution).astype(int)
        row_data["wcet"] = row_data["execution"] 
        # row_data["wcet"] = row_data["read"] + row_data["execution"] + row_data["write"]
        data_rows.append(row_data)
        
tasks_data = pd.DataFrame(data_rows, columns=column_names)
tasks_data

,name,core,period,wcet,read,execution,write
0,Task_1000ms,2,1000000,22,32,22,33
1,Task_200ms,2,200000,25,24,25,19
2,Task_100ms,2,100000,1421,174,1421,117
3,Task_50ms,2,50000,500,32,500,20
4,Task_20ms,2,20000,1570,199,1570,106
5,Task_10ms,3,10000,1739,273,1739,202
6,Angle_Sync,1,6660,578,130,578,74
7,Task_5ms,2,5000,165,7,165,4
8,Task_2ms,2,2000,72,9,72,2
9,Task_1ms,1,1000,127,20,127,6


In [12]:
for task_info in tasks_info:
    if task_info['task_name'] == "ISR_10":
        print(task_info["task_name"], len(task_info["phased_execution"]), len(task_info["inter_arrival_time_ns"]))
        print(task_info["lower_bound_ns"], task_info["upper_bound_ns"])
        
        print(task_info["inter_arrival_time_ns"][:10])



ISR_10 85714 85714
700000 800000
[703721, 763352, 737718, 797028, 735145, 789337, 712434, 743689, 786571, 711862]
